In [236]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import *
from sklearn.preprocessing import *
from sklearn.linear_model import *
from sklearn.metrics import *
from sklearn.ensemble import *
import warnings
warnings.filterwarnings("ignore")
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk
from imblearn.over_sampling import *
from scipy.stats import *

In [211]:
df = pd.read_csv("customer_booking.csv",encoding="ISO-8859-1")
df

,num_passengers,sales_channel,trip_type,purchase_lead,length_of_stay,flight_hour,flight_day,route,booking_origin,wants_extra_baggage,wants_preferred_seat,wants_in_flight_meals,flight_duration,booking_complete
0,2,Internet,RoundTrip,262,19,7,Sat,AKLDEL,New Zealand,1,0,0,5.52,0
1,1,Internet,RoundTrip,112,20,3,Sat,AKLDEL,New Zealand,0,0,0,5.52,0
2,2,Internet,RoundTrip,243,22,17,Wed,AKLDEL,India,1,1,0,5.52,0
3,1,Internet,RoundTrip,96,31,4,Sat,AKLDEL,New Zealand,0,0,1,5.52,0
4,2,Internet,RoundTrip,68,22,15,Wed,AKLDEL,India,1,0,1,5.52,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,2,Internet,RoundTrip,27,6,9,Sat,PERPNH,Australia,1,0,1,5.62,0
49996,1,Internet,RoundTrip,111,6,4,Sun,PERPNH,Australia,0,0,0,5.62,0
49997,1,Internet,RoundTrip,24,6,22,Sat,PERPNH,Australia,0,0,1,5.62,0
49998,1,Internet,RoundTrip,15,6,11,Mon,PERPNH,Australia,1,0,1,5.62,0


In [212]:
ohe = OneHotEncoder(drop="first")
orde = OrdinalEncoder()

#Sales Channel
sales_encode = ohe.fit(df[["sales_channel"]])
sales_channel = sales_encode.transform(df[["sales_channel"]]).toarray().astype("int")
print("Sales Channel   : ",sales_encode.categories_[0])

#Trip Type
trip_encode = ohe.fit(df[["trip_type"]])
trip_type = trip_encode.transform(df[["trip_type"]]).toarray().astype("int")
print("Trip Type       : ",trip_encode.categories_[0])

#Flight Day
flight_var = pd.DataFrame(df["flight_day"].map({"Sun":0,"Mon":1,"Tue":2,"Wed":3,"Thu":4,"Fri":5,"Sat":6}))
flight_encode = ohe.fit(flight_var[["flight_day"]])
flight_day =  flight_encode.transform(flight_var[["flight_day"]]).toarray().astype("int")
print("Flight Day      : ",flight_encode.categories_[0])

route_encode = orde.fit(df[["route"]])
route = route_encode.transform(df[["route"]]).astype("int")

booking_encode = orde.fit(df[["booking_origin"]])
booking_origin = booking_encode.transform(df[["booking_origin"]]).astype("int")

Sales Channel   :  ['Internet' 'Mobile']
Trip Type       :  ['CircleTrip' 'OneWay' 'RoundTrip']
Flight Day      :  [0 1 2 3 4 5 6]


In [213]:
df_1 = df.drop(["sales_channel","trip_type","flight_day","route","booking_origin","booking_complete"],axis=1,inplace=False)

In [214]:
df_1["sales_chanel"] = sales_channel
df_1["route"] = route
df_1["booking_origin"] = booking_origin
df_1[['OneWay','RoundTrip']] = pd.DataFrame(trip_type,columns=['OneWay','RoundTrip'])
df_1[["Mon","Tue","Wed","Thu","Fri","Sat"]] = pd.DataFrame(flight_day,columns=["Mon","Tue","Wed","Thu","Fri","Sat"])

In [215]:
df_1

,num_passengers,purchase_lead,length_of_stay,flight_hour,wants_extra_baggage,wants_preferred_seat,wants_in_flight_meals,flight_duration,sales_chanel,route,booking_origin,OneWay,RoundTrip,Mon,Tue,Wed,Thu,Fri,Sat
0,2,262,19,7,1,0,0,5.52,0,0,61,0,1,0,0,0,0,0,1
1,1,112,20,3,0,0,0,5.52,0,0,61,0,1,0,0,0,0,0,1
2,2,243,22,17,1,1,0,5.52,0,0,36,0,1,0,0,1,0,0,0
3,1,96,31,4,0,0,1,5.52,0,0,61,0,1,0,0,0,0,0,1
4,2,68,22,15,1,0,1,5.52,0,0,36,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,2,27,6,9,1,0,1,5.62,0,720,4,0,1,0,0,0,0,0,1
49996,1,111,6,4,0,0,0,5.62,0,720,4,0,1,0,0,0,0,0,0
49997,1,24,6,22,0,0,1,5.62,0,720,4,0,1,0,0,0,0,0,1
49998,1,15,6,11,1,0,1,5.62,0,720,4,0,1,1,0,0,0,0,0


In [216]:
help(RandomForestClassifier)

Help on class RandomForestClassifier in module sklearn.ensemble._forest:

class RandomForestClassifier(ForestClassifier)
 |  RandomForestClassifier(n_estimators=100, *, criterion='gini', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='sqrt', max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)
 |  
 |  A random forest classifier.
 |  
 |  A random forest is a meta estimator that fits a number of decision tree
 |  classifiers on various sub-samples of the dataset and uses averaging to
 |  improve the predictive accuracy and control over-fitting.
 |  The sub-sample size is controlled with the `max_samples` parameter if
 |  `bootstrap=True` (default), otherwise the whole dataset is used to build
 |  each tree.
 |  
 |  Read more in the :ref:`User Guide <forest>`.
 |  
 |  Parameters
 |  ----------


In [217]:
df_1

,num_passengers,purchase_lead,length_of_stay,flight_hour,wants_extra_baggage,wants_preferred_seat,wants_in_flight_meals,flight_duration,sales_chanel,route,booking_origin,OneWay,RoundTrip,Mon,Tue,Wed,Thu,Fri,Sat
0,2,262,19,7,1,0,0,5.52,0,0,61,0,1,0,0,0,0,0,1
1,1,112,20,3,0,0,0,5.52,0,0,61,0,1,0,0,0,0,0,1
2,2,243,22,17,1,1,0,5.52,0,0,36,0,1,0,0,1,0,0,0
3,1,96,31,4,0,0,1,5.52,0,0,61,0,1,0,0,0,0,0,1
4,2,68,22,15,1,0,1,5.52,0,0,36,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,2,27,6,9,1,0,1,5.62,0,720,4,0,1,0,0,0,0,0,1
49996,1,111,6,4,0,0,0,5.62,0,720,4,0,1,0,0,0,0,0,0
49997,1,24,6,22,0,0,1,5.62,0,720,4,0,1,0,0,0,0,0,1
49998,1,15,6,11,1,0,1,5.62,0,720,4,0,1,1,0,0,0,0,0


In [221]:
ss = StandardScaler()
scale = ss.fit(df_1)
df_2 = pd.DataFrame(scale.transform(df_1),columns=df_1.columns)

In [222]:
df_2

,num_passengers,purchase_lead,length_of_stay,flight_hour,wants_extra_baggage,wants_preferred_seat,wants_in_flight_meals,flight_duration,sales_chanel,route,booking_origin,OneWay,RoundTrip,Mon,Tue,Wed,Thu,Fri,Sat
0,0.400684,1.957530,-0.119353,-0.381764,0.703747,-0.649919,-0.863497,-1.174175,-0.355785,-1.724217,0.694118,-0.08832,0.100808,-0.439744,-0.425769,-0.425802,-0.417577,-0.395428,2.757334
1,-0.579559,0.299164,-0.089844,-1.120780,-1.420965,-0.649919,-0.863497,-1.174175,-0.355785,-1.724217,0.694118,-0.08832,0.100808,-0.439744,-0.425769,-0.425802,-0.417577,-0.395428,2.757334
2,0.400684,1.747470,-0.030824,1.465775,0.703747,1.538654,-0.863497,-1.174175,-0.355785,-1.724217,-0.069097,-0.08832,0.100808,-0.439744,-0.425769,2.348512,-0.417577,-0.395428,-0.362669
3,-0.579559,0.122272,0.234761,-0.936026,-1.420965,-0.649919,1.158082,-1.174175,-0.355785,-1.724217,0.694118,-0.08832,0.100808,-0.439744,-0.425769,-0.425802,-0.417577,-0.395428,2.757334
4,0.400684,-0.187290,-0.030824,1.096267,0.703747,-0.649919,1.158082,-1.174175,-0.355785,-1.724217,-0.069097,-0.08832,0.100808,-0.439744,-0.425769,2.348512,-0.417577,-0.395428,-0.362669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0.400684,-0.640577,-0.502977,-0.012257,0.703747,-0.649919,1.158082,-1.107368,-0.355785,1.443473,-1.046010,-0.08832,0.100808,-0.439744,-0.425769,-0.425802,-0.417577,-0.395428,2.757334
49996,-0.579559,0.288108,-0.502977,-0.936026,-1.420965,-0.649919,-0.863497,-1.107368,-0.355785,1.443473,-1.046010,-0.08832,0.100808,-0.439744,-0.425769,-0.425802,-0.417577,-0.395428,-0.362669
49997,-0.579559,-0.673744,-0.502977,2.389544,-1.420965,-0.649919,1.158082,-1.107368,-0.355785,1.443473,-1.046010,-0.08832,0.100808,-0.439744,-0.425769,-0.425802,-0.417577,-0.395428,2.757334
49998,-0.579559,-0.773246,-0.502977,0.357251,0.703747,-0.649919,1.158082,-1.107368,-0.355785,1.443473,-1.046010,-0.08832,0.100808,2.274053,-0.425769,-0.425802,-0.417577,-0.395428,-0.362669


In [250]:
df_3 = df_2.copy()
df_3["booking_complete"] = df["booking_complete"].copy()

In [251]:
df_3

,num_passengers,purchase_lead,length_of_stay,flight_hour,wants_extra_baggage,wants_preferred_seat,wants_in_flight_meals,flight_duration,sales_chanel,route,booking_origin,OneWay,RoundTrip,Mon,Tue,Wed,Thu,Fri,Sat,booking_complete
0,0.400684,1.957530,-0.119353,-0.381764,0.703747,-0.649919,-0.863497,-1.174175,-0.355785,-1.724217,0.694118,-0.08832,0.100808,-0.439744,-0.425769,-0.425802,-0.417577,-0.395428,2.757334,0
1,-0.579559,0.299164,-0.089844,-1.120780,-1.420965,-0.649919,-0.863497,-1.174175,-0.355785,-1.724217,0.694118,-0.08832,0.100808,-0.439744,-0.425769,-0.425802,-0.417577,-0.395428,2.757334,0
2,0.400684,1.747470,-0.030824,1.465775,0.703747,1.538654,-0.863497,-1.174175,-0.355785,-1.724217,-0.069097,-0.08832,0.100808,-0.439744,-0.425769,2.348512,-0.417577,-0.395428,-0.362669,0
3,-0.579559,0.122272,0.234761,-0.936026,-1.420965,-0.649919,1.158082,-1.174175,-0.355785,-1.724217,0.694118,-0.08832,0.100808,-0.439744,-0.425769,-0.425802,-0.417577,-0.395428,2.757334,0
4,0.400684,-0.187290,-0.030824,1.096267,0.703747,-0.649919,1.158082,-1.174175,-0.355785,-1.724217,-0.069097,-0.08832,0.100808,-0.439744,-0.425769,2.348512,-0.417577,-0.395428,-0.362669,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0.400684,-0.640577,-0.502977,-0.012257,0.703747,-0.649919,1.158082,-1.107368,-0.355785,1.443473,-1.046010,-0.08832,0.100808,-0.439744,-0.425769,-0.425802,-0.417577,-0.395428,2.757334,0
49996,-0.579559,0.288108,-0.502977,-0.936026,-1.420965,-0.649919,-0.863497,-1.107368,-0.355785,1.443473,-1.046010,-0.08832,0.100808,-0.439744,-0.425769,-0.425802,-0.417577,-0.395428,-0.362669,0
49997,-0.579559,-0.673744,-0.502977,2.389544,-1.420965,-0.649919,1.158082,-1.107368,-0.355785,1.443473,-1.046010,-0.08832,0.100808,-0.439744,-0.425769,-0.425802,-0.417577,-0.395428,2.757334,0
49998,-0.579559,-0.773246,-0.502977,0.357251,0.703747,-0.649919,1.158082,-1.107368,-0.355785,1.443473,-1.046010,-0.08832,0.100808,2.274053,-0.425769,-0.425802,-0.417577,-0.395428,-0.362669,0


In [252]:
sm = SMOTE()
x_bal,y_bal = sm.fit_resample(df_2,df["booking_complete"])

In [253]:
df_4 = x_bal.copy()
df_4["booking_complete"] = y_bal

In [258]:
for i in x_bal:
    pr = pearsonr(x_bal[i],y_bal)
    p_value = round(pr[1],2)
    if p_value > 0.05:
        print(i,":",p_value)

Mon : 0.16
Tue : 0.21
Thu : 0.06
Sat : 0.28


In [259]:
df_5 = df_4.drop(["Mon","Tue","Thu","Sat"],axis=1,inplace=False)
df_5

,num_passengers,purchase_lead,length_of_stay,flight_hour,wants_extra_baggage,wants_preferred_seat,wants_in_flight_meals,flight_duration,sales_chanel,route,booking_origin,OneWay,RoundTrip,Wed,Fri,booking_complete
0,0.400684,1.957530,-0.119353,-0.381764,0.703747,-0.649919,-0.863497,-1.174175,-0.355785,-1.724217,0.694118,-0.08832,0.100808,-0.425802,-0.395428,0
1,-0.579559,0.299164,-0.089844,-1.120780,-1.420965,-0.649919,-0.863497,-1.174175,-0.355785,-1.724217,0.694118,-0.08832,0.100808,-0.425802,-0.395428,0
2,0.400684,1.747470,-0.030824,1.465775,0.703747,1.538654,-0.863497,-1.174175,-0.355785,-1.724217,-0.069097,-0.08832,0.100808,2.348512,-0.395428,0
3,-0.579559,0.122272,0.234761,-0.936026,-1.420965,-0.649919,1.158082,-1.174175,-0.355785,-1.724217,0.694118,-0.08832,0.100808,-0.425802,-0.395428,0
4,0.400684,-0.187290,-0.030824,1.096267,0.703747,-0.649919,1.158082,-1.174175,-0.355785,-1.724217,-0.069097,-0.08832,0.100808,2.348512,-0.395428,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85039,-0.579559,-0.364183,0.391531,0.677601,0.703747,-0.649919,-0.863497,-1.474806,-0.355785,-0.172622,0.075151,-0.08832,0.100808,-0.425802,-0.395428,1
85040,0.174475,-0.685651,-0.546107,-0.168587,-1.420965,-0.649919,1.158082,-0.638176,-0.355785,0.027150,1.391578,-0.08832,0.100808,-0.425802,-0.395428,1
85041,1.078393,0.174552,-0.284395,-0.055587,0.703747,1.538654,-0.863497,1.037139,-0.355785,-1.697819,0.483053,-0.08832,0.100808,-0.425802,-0.395428,1
85042,0.400684,-0.139563,-0.543032,-0.718261,-1.420965,-0.649919,-0.863497,-0.185430,-0.355785,-0.203738,0.144603,-0.08832,0.100808,-0.425802,2.528904,1


In [280]:
X = df_5.drop("booking_complete",axis=1,inplace=False)
y = df_5["booking_complete"]

In [281]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=42)

In [282]:
rfc = RandomForestClassifier()
model = rfc.fit(X_train,y_train)

In [306]:
y_pred = model.predict(X_test)

In [317]:
c_matrix = pd.DataFrame(confusion_matrix(y_test,y_pred),columns=["Predicted Positive","Predicted Negative"],index=["Actual Positive","Actual Negative"])
acc_score = accuracy_score(y_test,y_pred)
class_report = classification_report(y_test,y_pred)

In [322]:
print(class_report)

              precision    recall  f1-score   support

           0       0.89      0.93      0.91     10545
           1       0.93      0.88      0.90     10716

    accuracy                           0.91     21261
   macro avg       0.91      0.91      0.91     21261
weighted avg       0.91      0.91      0.91     21261


In [325]:
print(mean_absolute_error(y_test,y_pred))
print(mean_squared_error(y_test,y_pred))
print(mean_absolute_percentage_error(y_test,y_pred))
print(mean_squared_log_error(y_test,y_pred))

0.09458633178119562
0.09458633178119562
157597390657243.3
0.04544428817974239
